In [46]:
from feature.clip_make_dataset import make_data
import pandas as pd
from glob import glob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import glob
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

In [47]:
train_input_list = './data/train_input/*.csv'
train_target_list = './data/train_target/*.csv'

test_input_list = './data/test_input/*.csv'
test_target_list = './data/test_target/*.csv'

In [48]:
train, test, p_train, p_test = make_data(train_input_list, train_target_list, test_input_list, test_target_list)

In [49]:
train_1 = pd.read_csv('./data/clip_preprocessing/clip_train_quantile0.25.csv')
test_1 = pd.read_csv('./data/clip_preprocessing/clip_test_quantile0.25.csv')
train_3 = pd.read_csv('./data/clip_preprocessing/clip_train_quantile0.75.csv')
test_3 = pd.read_csv('./data/clip_preprocessing/clip_test_quantile0.75.csv')
train_std = pd.read_csv('./data/clip_preprocessing/clip_train_std.csv')
test_std = pd.read_csv('./data/clip_preprocessing/clip_test_std.csv')
train_mid = pd.read_csv('./data/clip_preprocessing/clip_train_median.csv')
test_mid = pd.read_csv('./data/clip_preprocessing/clip_test_median.csv')

In [50]:
train_3.columns = [col+' q3' for col in train_3.columns]
test_3.columns = [col+' q3' for col in test_3.columns]
train_1.columns = [col+' q1' for col in train_1.columns]
test_1.columns = [col+' q1' for col in test_1.columns]
train_std.columns = [col+' std' for col in train_std.columns]
test_std.columns = [col+' std' for col in test_std.columns]
train_mid.columns = [col+' mid' for col in train_mid.columns]
test_mid.columns = [col+' mid' for col in test_mid.columns]

In [51]:
train = train.reset_index()
test  = test.reset_index()
train_3 = train_3.reset_index()
test_3 = test_3.reset_index()
train_1 = train_1.reset_index()
test_1 = test_1.reset_index()
train_std = train_std.reset_index()
test_std = test_std.reset_index()
train_mid = train_mid.reset_index()
test_mid = test_mid.reset_index()


In [52]:
train = pd.concat([train, train_1, train_3, train_std, train_mid], axis=1)
test = pd.concat([test, test_1, test_3, test_std, test_mid], axis=1)

train = train.drop(['index'], axis=1)
test = test.drop(['index'], axis=1)

train = pd.concat([train, p_train], axis=1)
test = pd.concat([test, p_test], axis=1)

In [44]:
def model_train(train, test, model):
    '''
    train shape = (784, 18), test shape = (140, 18)
    model => sklearn api model
    '''

    X = train.drop(['predicted_weight_g', 'Case', 'obs_time', '6time'], axis=1)
    y = train['predicted_weight_g']

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=113, shuffle=True)
    
    x_test = test.drop(['predicted_weight_g', 'Case', 'obs_time', '6time'], axis=1)
    
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    rmse = mean_squared_error(y_val, y_val_pred)**0.5
    print(f"validation rmse: {rmse}")
    
    x_train = train.drop(['predicted_weight_g', 'Case', 'obs_time', '6time'], axis=1)
    y_train = train['predicted_weight_g']
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    x_test['predicted_weight_g'] = y_pred
    submit = x_test[['DAT', 'predicted_weight_g']]
    submit['DAT'] = submit['DAT']+1
    all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
    for idx, test_path in enumerate(all_target_list):
        submit_df = pd.read_csv(test_path)
        submit_df['predicted_weight_g'] = submit['predicted_weight_g'][idx*28:idx*28+28].values
        submit_df.to_csv(test_path, index=False)

In [45]:
xgb = XGBRegressor()
cat = CatBoostRegressor(verbose=100, early_stopping_rounds=50)
lgbm = LGBMRegressor()
linear = LinearRegression()

model_train(train, test, cat)

Learning rate set to 0.037231
0:	learn: 40.6769718	total: 142ms	remaining: 2m 21s
100:	learn: 8.3643397	total: 765ms	remaining: 6.81s
200:	learn: 4.6210395	total: 1.34s	remaining: 5.32s
300:	learn: 3.0764467	total: 2.01s	remaining: 4.67s
400:	learn: 2.0788991	total: 2.95s	remaining: 4.41s
500:	learn: 1.5521974	total: 3.83s	remaining: 3.82s
600:	learn: 1.2182231	total: 4.76s	remaining: 3.16s
700:	learn: 0.9548928	total: 5.69s	remaining: 2.43s
800:	learn: 0.7456123	total: 6.39s	remaining: 1.59s
900:	learn: 0.5954728	total: 6.96s	remaining: 765ms
999:	learn: 0.4874976	total: 7.52s	remaining: 0us
validation rmse: 7.749375476747137
Learning rate set to 0.039399
0:	learn: 40.3156010	total: 6.27ms	remaining: 6.26s
100:	learn: 7.6637459	total: 687ms	remaining: 6.11s
200:	learn: 4.4279079	total: 1.36s	remaining: 5.39s
300:	learn: 3.0274731	total: 2s	remaining: 4.66s
400:	learn: 2.1201097	total: 2.77s	remaining: 4.14s
500:	learn: 1.5873376	total: 3.71s	remaining: 3.7s
600:	learn: 1.2656195	total